In [3]:
# What are the products and product groups with the best volumes, revenues, profits, transactions, customers, etc.?

In [2]:
All = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT *
    FROM CleanedData
    LIMIT 6;
    """
)

df_All = client.query(All).to_dataframe()  

df_All


,trans_id,trans_dt,store_id,cust_id,prod_id,sales_amt,sales_qty,sales_wgt,prod_category
0,200202010110400664,2020-02-02,1011,1123948922,20047449001,2.99,1,0.00,Vegetables
1,200202010110202416,2020-02-02,1011,1126052426,20302298003,3.99,1,0.00,Sweet
2,200131010112100153,2020-01-31,1011,1147407443,20162945001,4.99,1,0.00,Fruit
3,180119010230508490,2018-01-19,1023,1123875171,20357915,44.30,1,2.51,Fresh Beef
4,180119010230209223,2018-01-19,1023,1124234284,20005237,1.09,1,0.00,Canned
5,180120010230306191,2018-01-20,1023,1050396337,20946531,12.99,1,0.00,Pet Food & Supplies


In [4]:
# What are the products with the best volumes (quantity)?
a1 = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_id, SUM(sales_qty) as total_units_sold
    FROM CleanedData
    GROUP BY prod_id
    ORDER BY total_units_sold DESC
    LIMIT 10
    """
)

# What are the products with the best volumes (quantity)?
df_a1 = client.query(a1).to_dataframe()  

df_a1


,prod_id,total_units_sold
0,20175355001,13720283
1,21097012001,6818371
2,20047851,6355216
3,20123850,6085925
4,20028593001,6055142
5,20040489001,5468571
6,20070132001,5240164
7,20076950,4774431
8,20668578,4353108
9,20812144001,4152426


In [5]:
# What are the products with the best volumes (weight)?
a2 = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_id, SUM(sales_wgt) AS total_weight_sold
    FROM CleanedData
    GROUP BY prod_id
    ORDER BY total_weight_sold DESC
    LIMIT 10
    """
)

# What are the products with the best volumes (weight)?
df_a2 = client.query(a2).to_dataframe()  

df_a2


,prod_id,total_weight_sold
0,20175355001,13219141.36
1,20139509001,2478044.90
2,20425775001,2045943.10
3,20159199001,1919931.28
4,20426078001,1795505.24
5,20127708001,1615074.80
6,20026703001,1532684.46
7,20083526001,1462454.36
8,20426141001,1369735.68
9,20159690001,1104106.60


In [6]:
# What are the product groups with the best volumes (quantity)?
b1 = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_category, SUM(sales_qty) as total_units_sold
    FROM CleanedData
    GROUP BY prod_category
    ORDER BY total_units_sold DESC
    LIMIT 10
    """
)

# What are the product groups with the best volumes (quantity)?
df_b1 = client.query(b1).to_dataframe()  

df_b1


,prod_category,total_units_sold
0,Vegetables,122648828
1,Fruit,96113165
2,Snacks,56152072
3,Natural Foods,55434773
4,In-Store,45439939
5,Cold Beverages,42424542
6,Canned,40961901
7,Milk & Eggs,34638850
8,Meal Makers,33465730
9,Commercial,32018523


In [14]:
df_b1

,prod_category,total_units_sold
0,Vegetables,122648828
1,Fruit,96113165
2,Snacks,56152072
3,Natural Foods,55434773
4,In-Store,45439939
5,Cold Beverages,42424542
6,Canned,40961901
7,Milk & Eggs,34638850
8,Meal Makers,33465730
9,Commercial,32018523


In [7]:
# What are the product groups with the best volumes (weight)?
b2 = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_category, SUM(sales_wgt) AS total_weight_sold
    FROM CleanedData
    GROUP BY prod_category
    ORDER BY total_weight_sold DESC
    LIMIT 10
    """
)

# What are the product groups with the best volumes (weight)?
df_b2 = client.query(b2).to_dataframe()  

df_b2


,prod_category,total_weight_sold
0,Fruit,32499912.66
1,Vegetables,20022459.67
2,Fresh-Poultry,8617427.59
3,Fresh Beef,6392292.11
4,Fresh-Pork,2215236.32
5,Salad Bar,1535923.81
6,Deli Meat,1506839.74
7,Fresh Seafood,1299724.15
8,HMR,932988.05
9,Fresh-Lamb/Veal/Sausage,712270.27


In [8]:
# What are the products with the best revenues?
c = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_id, prod_category, SUM(sales_amt) as total_sales
    FROM CleanedData
    GROUP BY prod_id, prod_category
    ORDER BY total_sales DESC
    --LIMIT 10
    """
)

# What are the products with the best revenues?
df_c = client.query(c).to_dataframe()  

df_c.to_csv('prod_sales.csv', index=False)
df_c


,prod_id,prod_category,total_sales
0,20733932,Dispensing,1.221134e+08
1,20027156,Lottery - Electronic,2.056663e+07
2,20175355001,Fruit,1.930816e+07
3,20252014,HMR,1.493291e+07
4,20425775001,Fruit,1.428215e+07
...,...,...,...
134224,20043252,Optical,-1.871894e+06
134225,20534096,Lottery - Electronic,-2.074538e+06
134226,20650081,Coupons,-3.241598e+06
134227,20527440,Coupons,-5.616861e+06


In [9]:
# What are the product category with the best revenues?
d = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_category, SUM(sales_amt) as total_sales
    FROM CleanedData
    GROUP BY prod_category
    ORDER BY total_sales DESC
    --LIMIT 10
    """
)

# What are the product groups with the best revenues?
df_d = client.query(d).to_dataframe()  

df_d.to_csv('prod_cate_sales.csv', index=False)
df_d


,prod_category,total_sales
0,Vegetables,4.096608e+08
1,Fruit,3.138898e+08
2,Natural Foods,2.904772e+08
3,Snacks,1.712150e+08
4,HMR,1.620010e+08
...,...,...
90,Cosmetic Treatments,1.118520e+03
91,Cosmetic Fragrances,8.483500e+02
92,Jewelery & Accessories,7.928900e+02
93,Supplies,7.513100e+02


In [10]:
# What are the products with the best transactions, etc.?
e = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_id, COUNT(trans_id) as total_prod_trans
    FROM CleanedData
    GROUP BY prod_id
    ORDER BY total_prod_trans DESC
    LIMIT 10
    """
)

# What are the products with the best transactions, etc.?
df_e = client.query(e).to_dataframe()  

df_e


,prod_id,total_prod_trans
0,20175355001,13720294
1,20668578,4348227
2,20070132001,4242970
3,20812144001,3574510
4,20145621001,3102016
5,20007535001,3042138
6,21097012001,3035599
7,20527440,2934861
8,20671789,2920847
9,20028593001,2914054


In [11]:
# What are the product groups with the best transactions, etc.?
f = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_category, COUNT(trans_id) as total_prod_cate_trans
    FROM CleanedData
    GROUP BY prod_category
    ORDER BY total_prod_cate_trans DESC
    LIMIT 10
    """
)

# What are the product groups with the best transactions?
df_f = client.query(f).to_dataframe()  

df_f


,prod_category,total_prod_cate_trans
0,Vegetables,112446078
1,Fruit,76951653
2,Snacks,46568633
3,Natural Foods,44251003
4,In-Store,32576304
5,Cold Beverages,30781675
6,Milk & Eggs,30448381
7,Commercial,27714876
8,Meal Makers,25789376
9,Cheese/Butter/Margarine,23847899


In [12]:
# What are the products with the best customers, etc.?
g = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_id, COUNT(cust_id) as total_cust_cnt
    FROM CleanedData
    GROUP BY prod_id
    ORDER BY total_cust_cnt DESC
    LIMIT 10
    """
)

# What are the products with the best customers, etc.?
df_g = client.query(g).to_dataframe()  

df_g


,prod_id,total_cust_cnt
0,20175355001,13720297
1,20668578,4348230
2,20070132001,4242971
3,20812144001,3574510
4,20145621001,3102017
5,20007535001,3042138
6,21097012001,3035599
7,20527440,2934861
8,20671789,2920847
9,20028593001,2914054


In [13]:
# What are the product groups with the best customers, etc.?
h = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT prod_category, COUNT(cust_id) as total_cust_cnt
    FROM CleanedData
    GROUP BY prod_category
    ORDER BY total_cust_cnt DESC
    LIMIT 10
    """
)

# What are the product groups with the best customers, etc.?
df_h = client.query(h).to_dataframe()  

df_h


,prod_category,total_cust_cnt
0,Vegetables,112446055
1,Fruit,76951642
2,Snacks,46568634
3,Natural Foods,44250995
4,In-Store,32576295
5,Cold Beverages,30781667
6,Milk & Eggs,30448371
7,Commercial,27714869
8,Meal Makers,25789370
9,Cheese/Butter/Margarine,23847899
